## 1. 전처리

1. 각 매장 마다의 리뷰 리스트를 나누고
2. null값 확인
3. 한글빼고 삭제
4. 중복제거
5. 길이 10이하 텍스트 제거
5. 레이블링

In [1]:
import re
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Okt
#from pykospacing import Spacing
from collections import Counter

In [2]:
import numpy as np
import pandas as pd

In [3]:
import json

with open('./naver_횟집_by_metropolitan.json','r',encoding='utf-8') as f:
    data = json.load(f)

In [4]:
#매장의 개수
len(data)

1688

In [5]:
#1. 각 매장 마다의 리뷰 리스트를 나누고
total_review = list()
import pandas as pd
for store_info in data.values():
    total_review.extend(store_info['review'])

In [6]:
# 전체 리뷰 개수
len(total_review)

155212

In [7]:
#데이터 프레임 생성
df = pd.DataFrame(total_review, columns=['Review'])

In [8]:
df

,Review
0,지금까지 다녀봤던 참치집 중 부위가 가장 좋았던거 같아요! 어묵탕 샐러드 생선조림 ...
1,처음 들어갈땐 사장님이 계신거같더니 먹는 중에는 퇴근하셨나? 벨을 눌러도 안오네요ㅋ...
2,가족들 다 참치회를 좋아해서 생일겸 맛있는 참치회 먹으려고 집근처로 찾아보던중에 알...
3,일등참치 맛집참치 가성비참치 최고참치 증독참치 구청뒤쪽에서 신설동으로 계속 딸라다닐...
4,음식이 깔끔하고 맛있네요
...,...
155207,깨끗하고 친절해요 맘에듭니다
155208,친절해요
155209,깨끗하고 친절해요 맘에듭니다
155210,친절해요


In [9]:
# null값 확인
df.isnull().sum()

Review    0
dtype: int64

In [10]:
#여러 공백을 하나의 공백으로 줄인다.
import re
def apply_regular_expreesion(text):
    hangul = re.compile('[^ 가-힣]')
    #한글이 아닌 문자 제거
    result = hangul.sub('', text) 
    #1개 이상의 공백 -> 1개의 공백
    result = re.sub(' +',' ', result)
    return result

df['Review'] = df['Review'].apply(lambda x : apply_regular_expreesion(x))

In [11]:
tmp = df[df['Review']=='']
tmp

,Review
29,
102,
114,
128,
130,
...,...
155163,
155176,
155180,
155181,


In [12]:
#빈 문자열 -> NaN 치환
df['Review'].replace('', np.nan, inplace=True)
# null값 확인
print(df.isnull().sum())
df.dropna(inplace=True)
len(df)

Review    5223
dtype: int64


149989

In [13]:
df.duplicated(['Review']).sum()

54293

In [14]:
#중복 제거
df = df.drop_duplicates(['Review'])
print(df.duplicated(['Review']).sum())

0


In [15]:
print(df.isnull().sum())

Review    0
dtype: int64


In [16]:
len(df)

95696

In [17]:
# 10개 이하 시퀀스 제거
df = df[df['Review'].map(lambda x : len(str(x)) > 10) ]

In [18]:
df = df.reset_index(drop=True)

In [19]:
df

,Review
0,지금까지 다녀봤던 참치집 중 부위가 가장 좋았던거 같아요 어묵탕 샐러드 생선조림 순...
1,처음 들어갈땐 사장님이 계신거같더니 먹는 중에는 퇴근하셨나 벨을 눌러도 안오네요 보...
2,가족들 다 참치회를 좋아해서 생일겸 맛있는 참치회 먹으려고 집근처로 찾아보던중에 알...
3,일등참치 맛집참치 가성비참치 최고참치 증독참치 구청뒤쪽에서 신설동으로 계속 딸라다닐...
4,음식이 깔끔하고 맛있네요
...,...
83564,방어 넘 양도 많고 짱이네요포장도 갠찬아요
83565,횟감이 신선하고 쫀득한
83566,사장님 친절하시고 음식도맛있고 좋아요서비스도 엄청 좋아요
83567,깨끗하고 친절해요 맘에듭니다


# 2. 레이블링
긍정 - 1
부정 - 0 / 긍정 + 부정이면 부정으로 취급
관계없는 시퀀스 - 2 이상의 수

In [10]:
#현재 인덱스 2500까지 했다.
for i in range(df['Label'].count(), 2500):
    print(f'{i}번째 인덱스')
    print(df.loc[i,'Review'])
    label = input("레이블 입력하세요: ")
    label = int(label)
    df.loc[i,'Label'] = label

2993번째 인덱스
나오는 기타 요리들도 너무 맛있고 무엇보다 대게가 좋습니다 찌시는 방법도 직접 솥에 쪄주셔서 정성이 들어가 더 맛있는 것 같아요 너무 친절하셔서 
레이블 입력하세요: 1
2994번째 인덱스
너무 맛있게 잘 먹었습니다 사장님도 친절하시고 무엇보다 킹크랩이 엄청 살도 꽉차고 진짜 진짜 맛있습니다 무조건 재방문각입니다
레이블 입력하세요: 1
2995번째 인덱스
동교동 창천동 모두 먹었던 곳에서 제일 만족한 곳이에요 서비스는 물론이고 기본 채소 퀄리티부터가 정말 진짜 추천 많이합니다 기분좋아지는 것
레이블 입력하세요: 1
2996번째 인덱스
제가 사진찍는 기술이 후져서 실제 나온 것보다 못찍어서 아쉽네요 잘먹었습니다
레이블 입력하세요: 1
2997번째 인덱스
대게맛의 진수 서비스도 많고 정말 맛있습니다
레이블 입력하세요: 1
2998번째 인덱스
먹기 좋게 손쉽게 손질되어 나오고 속살도 꽉 차서 진짜 맛있게 잘 먹었습니다신선해서 그런지 비린내도 전혀 안났어요
레이블 입력하세요: 1
2999번째 인덱스
참 맛있게 먹고 갑니다
레이블 입력하세요: 1


In [10]:
# df_tmp 저장
import pickle
with open('df_tmp.pickle', 'rb') as f:
    df = pickle.load(f)

In [11]:
df

,Review,Label
0,지금까지 다녀봤던 참치집 중 부위가 가장 좋았던거 같아요 어묵탕 샐러드 생선조림 순...,1.0
1,처음 들어갈땐 사장님이 계신거같더니 먹는 중에는 퇴근하셨나 벨을 눌러도 안오네요 보...,0.0
2,가족들 다 참치회를 좋아해서 생일겸 맛있는 참치회 먹으려고 집근처로 찾아보던중에 알...,1.0
3,일등참치 맛집참치 가성비참치 최고참치 증독참치 구청뒤쪽에서 신설동으로 계속 딸라다닐...,1.0
4,음식이 깔끔하고 맛있네요,1.0
...,...,...
83564,방어 넘 양도 많고 짱이네요포장도 갠찬아요,NaN
83565,횟감이 신선하고 쫀득한,NaN
83566,사장님 친절하시고 음식도맛있고 좋아요서비스도 엄청 좋아요,NaN
83567,깨끗하고 친절해요 맘에듭니다,NaN


In [16]:
import pandas as pd
import numpy as np
df.value_counts(['Label']).sum()

2500

In [32]:
# 5000개의 데이터 랜덤 추출
tmp = df.sample(n=5000, replace = False, random_state=24)

In [33]:
tmp.count()

Review    5000
Label      150
dtype: int64

In [38]:
# 기존의 순차적 레이블링 진행했던 2500개의 레코드 df
# 새롭게 비복원 추출로 랜덤하게 뽑은 5000개 tmp를 합친다.
df = pd.concat([df.iloc[:2500,:], tmp], ignore_index=True)

In [39]:
df.count()

Review    7500
Label     2650
dtype: int64

In [41]:
# 중복제거
df.drop_duplicates(['Review'],ignore_index=True, inplace=True)
df.count()

Review    7350
Label     2500
dtype: int64

In [79]:
for i in range(df['Label'].count(), 3000):
    print(f'{i}번째 인덱스')
    print(df.loc[i,'Review'])
    label = input("레이블 입력하세요: ")
    label = int(label)
    df.loc[i,'Label'] = label

2940번째 인덱스
반찬들 하나하나 다 잘나왔어요 청어구이보다는 꽁 치가 더 좋을것 같긴 한데 방어 먹으러 왔는데 메뉴에 없는 방어참가자미 반반으로 해주셔
레이블 입력하세요: 1
2941번째 인덱스
방어우럭 포장했는데 광어우럭이 있네요 바쁠때 아니었는데 포장하기전에 확인해주시면 좋겠어요
레이블 입력하세요: 1
2942번째 인덱스
회가 너무 신선하고 맛있었어요
레이블 입력하세요: 1
2943번째 인덱스
혼밥 해서 죄송하지만 청결하고 친절하고음식도 신선하고 너무 좋아요 또갈게요
레이블 입력하세요: 1
2944번째 인덱스
위생 상태도 너무 좋고 맛있게 잘 먹었어요 그날 하 루 배가 터질 지경이었어요 상다리가 부러지게 나와서 회 상태도 너무 신선하고 좋더라
레이블 입력하세요: 1
2945번째 인덱스
항상친절하고 좋아요 노량진 갈때마다 무조건 청해진
레이블 입력하세요: 1
2946번째 인덱스
모듬회하고 물회 너무 맛있게 먹었어요초밥도 팔던데 다음에 재방문해서 먹어보려구요
레이블 입력하세요: 1
2947번째 인덱스
너무 맛있게 먹고 갓네요 서비스나 회 신선도나친절함이나 최고에요
레이블 입력하세요: 1
2948번째 인덱스
룸으로 배정 받아서 더 좋은시간 보냈네요
레이블 입력하세요: 1
2949번째 인덱스
맛있어요 점심특선 라멘은 좀 그랬어요
레이블 입력하세요: 1
2950번째 인덱스
차 차로 오는 사람들이 많다보니 많이 시끄럽고 정신이 없어요 
레이블 입력하세요: 0
2951번째 인덱스
자주가는 참치집입니다 맛 친절 양 질 다 좋아용
레이블 입력하세요: 1
2952번째 인덱스
최고였네요 싱싱하고 가성비 좋았습니다 기분 좋게 먹고 나왔어요 친구들에게도 추천하고 싶은 집입니다
레이블 입력하세요: 1
2953번째 인덱스
광어포차 좋아요친절하고 맛있어요
레이블 입력하세요: 1
2954번째 인덱스
회랑 새우튀김도 매운탕도 맛있었어요
레이블 입력하세요: 1
2955번째 인덱스
맛있고 가격도 좋아요
레이블 입력하세요: 1
2956번째 인덱스
여러가지 종류의 회와 해산물이 들어있어서 여

In [84]:
df.value_counts(['Label']).sum()

3000

In [87]:
result = df.iloc[:3000,:]
result

,Review,Label
0,지금까지 다녀봤던 참치집 중 부위가 가장 좋았던거 같아요 어묵탕 샐러드 생선조림 순...,1.0
1,처음 들어갈땐 사장님이 계신거같더니 먹는 중에는 퇴근하셨나 벨을 눌러도 안오네요 보...,0.0
2,가족들 다 참치회를 좋아해서 생일겸 맛있는 참치회 먹으려고 집근처로 찾아보던중에 알...,1.0
3,일등참치 맛집참치 가성비참치 최고참치 증독참치 구청뒤쪽에서 신설동으로 계속 딸라다닐...,1.0
4,음식이 깔끔하고 맛있네요,1.0
...,...,...
2995,사시미 종류도 많고 양도 좋아용,1.0
2996,절대 가지마세요 점도 못줌 성남동 관평동 용문동 에서 자주 애용하면서 체인점이라도 ...,0.0
2997,양도 엄청 많고 맛잇어용,1.0
2998,너무 맛있어요 사장님도 친절하셔서 기분 좋게 먹다갑니다,1.0


In [93]:
result = result.astype({'Label':'int'})

In [94]:
result

,Review,Label
0,지금까지 다녀봤던 참치집 중 부위가 가장 좋았던거 같아요 어묵탕 샐러드 생선조림 순...,1
1,처음 들어갈땐 사장님이 계신거같더니 먹는 중에는 퇴근하셨나 벨을 눌러도 안오네요 보...,0
2,가족들 다 참치회를 좋아해서 생일겸 맛있는 참치회 먹으려고 집근처로 찾아보던중에 알...,1
3,일등참치 맛집참치 가성비참치 최고참치 증독참치 구청뒤쪽에서 신설동으로 계속 딸라다닐...,1
4,음식이 깔끔하고 맛있네요,1
...,...,...
2995,사시미 종류도 많고 양도 좋아용,1
2996,절대 가지마세요 점도 못줌 성남동 관평동 용문동 에서 자주 애용하면서 체인점이라도 ...,0
2997,양도 엄청 많고 맛잇어용,1
2998,너무 맛있어요 사장님도 친절하셔서 기분 좋게 먹다갑니다,1


In [98]:
result['Label'].values

array([1, 0, 1, ..., 1, 1, 1])

In [101]:
# 관계없는 리뷰 삭제
drop_index = result[result['Label'] > 1].index
result.drop(drop_index,inplace=True)

In [102]:
result.reset_index(drop=True,inplace=True)

In [103]:
result.value_counts(['Label'])

Label
1        2405
0         527
dtype: int64

In [105]:
#객체 저장
import pickle
with open('result.pickle', 'wb') as f:
    pickle.dump(result,f)